In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,trp,trp
band,[9],7
commitnumber,1013f91,5014a19
conc,-,3.2e-07
dv,-,0.001
klin,-,2.22e-20
molecule,{'n2o': 3.2e-07},n2o
ng_adju,-,"[0, 0]"
ng_refs,-,"[2, 2]"
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.032298                    1 -0.008917
0.000750        2 -0.032010                    2 -0.008769
0.001052        3 -0.031390                    3 -0.008365
0.001476        4 -0.030751                    4 -0.007994
0.002070        5 -0.030103                    5 -0.007604
0.002904        6 -0.029455                    6 -0.007220
0.004074        7 -0.028838                    7 -0.006814
0.005714        8 -0.028230                    8 -0.006404
0.008015        9 -0.027651                    9 -0.005993
0.011243       10 -0.027118                   10 -0.005567
0.015771       11 -0.026649                   11 -0.005147
0.022122       12 -0.026272                   12 -0.004708
0.031031       13 -0.026033                   13 -0.004280
0.043528       14 -0.025768                   14 -0.003709
0.061057       15 -0.022985                   15 -0.001518
0.085645       16 -0.015724                   16  0.003487
0.120136       17 -0.005398                   17  0.010621
0.168516       18  0.006972                   18  0.019399
0.236378       19  0.021665                   19  0.030088
0.331549       20  0.039076                   20  0.043048
0.465100       21  0.059776                   21  0.058677
0.652400       22  0.081677                   22  0.075371
0.915100       23  0.086605                   23  0.081373
1.283650       24  0.073344                   24  0.074841
1.800600       25  0.056183                   25  0.063908
2.525700       26  0.038399                   26  0.050306
3.542800       27  0.024913                   27  0.037457
4.969550       28  0.016061                   28  0.026332
6.970850       29  0.010468                   29  0.016964
9.778100       30  0.006827                   30  0.009345
13.715850      31  0.004124                   31  0.003369
19.239350      32  0.001720                   32 -0.001303
26.987250      33 -0.000707                   33 -0.005276
37.855300      34 -0.003374                   34 -0.009346
53.100050      35 -0.007695                   35 -0.015046
73.887500      36 -0.013542                   36 -0.022184
97.662500      37 -0.018833                   37 -0.028287
121.437500     38 -0.021666                   38 -0.031639
145.212500     39 -0.022386                   39 -0.032881
168.987500     40 -0.023030                   40 -0.033589
192.762500     41 -0.023398                   41 -0.033813
216.537500     42 -0.023457                   42 -0.033613
240.312500     43 -0.023190                   43 -0.032979
264.087500     44 -0.022634                   44 -0.031969
287.862500     45 -0.021810                   45 -0.030616
311.637500     46 -0.020737                   46 -0.028940
335.412500     47 -0.019431                   47 -0.027002
359.187500     48 -0.017907                   48 -0.024789
382.962500     49 -0.016173                   49 -0.022348
406.737500     50 -0.014238                   50 -0.019691
430.512500     51 -0.012115                   51 -0.016839
454.287500     52 -0.009810                   52 -0.013818
478.062500     53 -0.007329                   53 -0.010598
501.837500     54 -0.004674                   54 -0.007246
525.612500     55 -0.001845                   55 -0.003710
549.387500     56  0.001158                   56 -0.000062
573.162500     57  0.004339                   57  0.003743
596.937500     58  0.007702                   58  0.007658
620.712500     59  0.011240                   59  0.011683
644.487500     60  0.014923                   60  0.015781
668.262500     61  0.018638                   61  0.019859
692.037500     62  0.022103                   62  0.023722
715.812500     63  0.024932                   63  0.027090
739.587500     64  0.026982                   64  0.029910
763.362500     65  0.028583                   65  0.032366
78

# Fluxes by Layer

CRD                              CLIRAD  \
                        flug          fldg      fnetg       flug   
pressure    level                                                  
0.000000    1     -23.524379  0.000000e+00 -23.524379 -23.303130   
0.000624    2     -23.524382  9.711183e-07 -23.524381 -23.303131   
0.000876    3     -23.524384  1.366855e-06 -23.524382 -23.303131   
0.001229    4     -23.524386  1.935221e-06 -23.524384 -23.303132   
0.001723    5     -23.524388  2.752090e-06 -23.524385 -23.303133   
0.002417    6     -23.524392  3.925857e-06 -23.524388 -23.303134   
0.003391    7     -23.524397  5.611987e-06 -23.524391 -23.303136   
0.004757    8     -23.524404  8.029005e-06 -23.524396 -23.303138   
0.006672    9     -23.524413  1.149243e-05 -23.524402 -23.303142   
0.009359    10    -23.524428  1.645010e-05 -23.524411 -23.303147   
0.013128    11    -23.524447  2.353729e-05 -23.524424 -23.303153   
0.018415    12    -23.524473  3.365170e-05 -23.524440 -23.303162   
0.025830    13    -23.524511  4.805734e-05 -23.524463 -23.303175   
0.036232    14    -23.524564  6.852260e-05 -23.524495 -23.303193   
0.050823    15    -23.524637  9.769785e-05 -23.524539 -23.303217   
0.071291    16    -23.524738  1.425094e-04 -23.524596 -23.303248   
0.100000    17    -23.524867  2.185849e-04 -23.524649 -23.303284   
0.140271    18    -23.525026  3.508232e-04 -23.524674 -23.303317   
0.196760    19    -23.525206  5.780893e-04 -23.524628 -23.303334   
0.275997    20    -23.525388  9.638771e-04 -23.524425 -23.303309   
0.387100    21    -23.525520  1.609982e-03 -23.523910 -23.303188   
0.543100    22    -23.525484  2.678640e-03 -23.522806 -23.302871   
0.761700    23    -23.525070  4.379436e-03 -23.520691 -23.302201   
1.068500    24    -23.524226  6.683312e-03 -23.517543 -23.301124   
1.498800    25    -23.523103  9.298232e-03 -23.513805 -23.299722   
2.102400    26    -23.521779  1.199190e-02 -23.509787 -23.298047   
2.949000    27    -23.520422  1.448504e-02 -23.505937 -23.296221   
4.136600    28    -23.519129  1.669671e-02 -23.502432 -23.294343   
5.802500    29    -23.517977  1.871511e-02 -23.499263 -23.292521   
8.139200    30    -23.517094  2.072873e-02 -23.496365 -23.290960   
11.417000   31    -23.516694  2.298007e-02 -23.493714 -23.290026   
16.014700   32    -23.517168  2.569978e-02 -23.491469 -23.290417   
22.464000   33    -23.519211  2.905705e-02 -23.490154 -23.293411   
31.510500   34    -23.524052  3.314045e-02 -23.490911 -23.301395   
44.200100   35    -23.533878  3.789516e-02 -23.495983 -23.318723   
62.000000   36    -23.554327  4.211926e-02 -23.512207 -23.354304   
85.775000   37    -23.594975  4.462884e-02 -23.550346 -23.420180   
109.550000  38    -23.650334  4.694789e-02 -23.603386 -23.502941   
133.325000  39    -23.717835  5.343116e-02 -23.664403 -23.597824   
157.100000  40    -23.793938  6.648678e-02 -23.727451 -23.701052   
180.875000  41    -23.877642  8.533237e-02 -23.792310 -23.811156   
204.650000  42    -23.968071  1.098656e-01 -23.858206 -23.926933   
228.425000  43    -24.064420  1.401519e-01 -23.924267 -24.047377   
252.200000  44    -24.165847  1.762675e-01 -23.989579 -24.171464   
275.975000  45    -24.271639  2.183145e-01 -24.053324 -24.298314   
299.750000  46    -24.381137  2.663887e-01 -24.114749 -24.427134   
323.525000  47    -24.493721  3.205702e-01 -24.173151 -24.557156   
347.300000  48    -24.608798  3.809235e-01 -24.227874 -24.687775   
371.075000  49    -24.725802  4.474967e-01 -24.278305 -24.818351   
394.850000  50    -24.844163  5.203107e-01 -24.323852 -24.948316   
418.625000  51    -24.963316  5.993657e-01 -24.363951 -25.077138   
442.400000  52    -25.082739  6.846685e-01 -24.398071 -25.204335   
466.175000  53    -25.201915  7.762148e-01 -24.425700 -25.329447   
489.950000  54    -25.320332  8.739897e-01 -24.446342 -25.451975   
513.725000  55    -25.437476  9.779698e-01 -24.459505 -25.571515   
537.500000  56    -25.552826  1.088125e+00 -24.464703 -25.687586   
561.275000  5

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')